In [1]:
import struct

# Constants as defined by the MD5 algorithm
S = [7, 12, 17, 22] * 4 + [5, 9, 14, 20] * 4 + [4, 11, 16, 23] * 4 + [6, 10, 15, 21] * 4
K = [int(abs(struct.unpack('<i', struct.pack('<f', 2 ** 32 * abs(i)))[:1][0]) % (2 ** 32)) for i in range(64)]

# Left rotation function
def left_rotate(x, amount):
    x &= 0xFFFFFFFF
    return ((x << amount) | (x >> (32 - amount))) & 0xFFFFFFFF

# Main MD5 algorithm function
def md5(message):
    # Padding the message
    message = bytearray(message.encode('utf-8'))  # Convert to byte array
    original_length = (8 * len(message)) & 0xFFFFFFFFFFFFFFFF  # Message length in bits
    message.append(0x80)  # Append '1' bit (10000000 in binary)

    while (len(message) * 8) % 512 != 448:
        message.append(0)  # Pad with '0's until length is 448 modulo 512

    message += struct.pack('<Q', original_length)  # Append original message length

    # Initialize MD5 buffers (A, B, C, D) to constants
    A = 0x67452301
    B = 0xefcdab89
    C = 0x98badcfe
    D = 0x10325476

    # Process message in 512-bit chunks
    for chunk_offset in range(0, len(message), 64):
        a, b, c, d = A, B, C, D
        chunk = message[chunk_offset:chunk_offset + 64]
        X = struct.unpack('<16I', chunk)  # Break chunk into sixteen 32-bit little-endian words

        # MD5 main loop
        for i in range(64):
            if 0 <= i <= 15:
                F = (b & c) | (~b & d)
                g = i
            elif 16 <= i <= 31:
                F = (d & b) | (~d & c)
                g = (5 * i + 1) % 16
            elif 32 <= i <= 47:
                F = b ^ c ^ d
                g = (3 * i + 5) % 16
            else:
                F = c ^ (b | ~d)
                g = (7 * i) % 16

            F = (F + a + K[i] + X[g]) & 0xFFFFFFFF
            a = d
            d = c
            c = b
            b = (b + left_rotate(F, S[i])) & 0xFFFFFFFF

        # Add this chunk's hash to the result so far
        A = (A + a) & 0xFFFFFFFF
        B = (B + b) & 0xFFFFFFFF
        C = (C + c) & 0xFFFFFFFF
        D = (D + d) & 0xFFFFFFFF

    # Produce the final hash (A, B, C, D are concatenated in little-endian)
    return struct.pack('<4I', A, B, C, D).hex()

# Example usage
input_string = "Hello, World!"
md5_hash_value = md5(input_string)

print(f"MD5 hash of '{input_string}' is: {md5_hash_value}")


MD5 hash of 'Hello, World!' is: 65df83f7098fee59610faba72193d25d


In [3]:
import hashlib

def computeMD5Hash(input_string):
    """
    Compute the MD5 hash of a given input string.
    
    Parameters:
    input_string (str): The input string to hash.
    
    Returns:
    str: The resulting MD5 hash in hexadecimal format.
    """
    # Step 1: Create an MD5 hash object
    md5_hash = hashlib.md5()
    
    # Step 2: Update the hash object with the bytes of the input string
    md5_hash.update(input_string.encode('utf-8'))
    
    # Step 3: Return the hexadecimal representation of the hash
    return md5_hash.hexdigest()

# Example usage
input_string = "Hello, World!"
md5_hash_value = computeMD5Hash(input_string)

print(f"MD5 hash of '{input_string}' is: {md5_hash_value}")


MD5 hash of 'Hello, World!' is: 65a8e27d8879283831b664bd8b7f0ad4
